In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql.window import *
import pymysql

In [2]:
connection = pymysql.connect(
        host='127.0.0.1',
        user='root',
        password='root',
        database='extenso_config'
    )

In [3]:
spark = SparkSession.builder.appName("hadoop_add_file").config("spark.jars", "C:\spark-3.5.1-bin-hadoop3\jars\mysql-connector-j-8.4.0.jar").getOrCreate()

In [4]:
url = "jdbc:mysql://localhost:3306/extenso_config"
properties = {
    "user": "root",
    "password": "root",
    "driver": "com.mysql.jdbc.Driver"
}

In [5]:
def hdfs_file_upload(url,properties,table_name):
     df = spark.read.jdbc(url=url, table=table_name, properties=properties)
     for i in range(df.count()):
         active_status = df.select("active_status").collect()
         status = active_status[i]['active_status']
         if status == False:
             with connection.cursor() as cursor:
                 update_query = f'UPDATE extenso_config.my_config set active_status = 1 where id = {i + 1}'
                 cursor.execute(update_query)
             schemas = df.select("schemaName").collect()
             schema = schemas[i]["schemaName"]
             tables_name = df.select("tableName").collect()
             name = tables_name[i]['tableName']
             locations = df.select("location").collect()
             location = locations[i]['location']
             hdfs_file_name = df.select("hdfs_file_name").collect()
             file_name = hdfs_file_name[i]['hdfs_file_name']
             jdbc_url = f"jdbc:mysql://localhost:3306/{schema}"
             connection_properties = {"user":"root", "password":"root", "driver":"com.mysql.jdbc.Driver"}
             dfs = spark.read.jdbc(url=jdbc_url, table = name, properties=connection_properties)
             hdfs_path = f'{location}/{file_name}'
             dfs.write.mode("overwrite").parquet(hdfs_path)
             
                 

In [6]:
hdfs_file_upload(url,properties,"my_config")

In [7]:
data = spark.read.parquet('hdfs://localhost:19000/anurag/sample_data')

In [8]:
data.show()